In [3]:
###############################################################################
# Import packages
###############################################################################
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import skew

# Load functions
from data_gen_functions import distance, fun_convert_time, fun_save_file, fun_load_file
from data_gen_functions import plot_instance
from data_gen_functions import fun_multi_dbscan, fun_cluster_features

###############################################################################
# Load and preprocess the data
###############################################################################
# Assign string "TSP" or "CVRP" to the following variable to define the routing problem
routing_problem = "CVRP"

if (routing_problem == "TSP"):
    # Get the name of the folder and the file name
    folder = "01_TSP"
    file_name = "tsp_instances_j"
    
    # Load data
    data_original = fun_load_file(subfolder_path="..\\..\\01_data\\" + folder, name=file_name + ".xlsx")
    
    # Remove first example instance before selecting columns that will not be changed
    data_original = data_original[data_original["Instance_id"] != 0].reset_index(drop = True)
    data = data_original[["X", "Y", "Marginal_Cost", "Savings", "Shapley Value", "Total cost", "Number Customers", "X_Depot", "Y_Depot", 
                          "Instance_id", "SHAPO", "Percentage_error"]]
    
    # Modify column names
    data = data.rename(columns={"Marginal_Cost": "Marginal Cost", "X_Depot": "X Depot", "Y_Depot": "Y Depot", "Total cost": "Total Cost", 
                                "Instance_id": "Instance ID", "Percentage_error": "Percentage Error"})
    
    # Reorder columns
    columns = ["Instance ID", "Number Customers", "X", "Y", "X Depot", "Y Depot", "Savings", "Marginal Cost", "Total Cost", 
               "Shapley Value", "SHAPO", "Percentage Error"]
    data = data[columns]

elif (routing_problem == "CVRP"):
    # Get the name of the folder and the file name
    folder = "02_CVRP"
    file_name = "cvrp_instances_j"
    
    # Load data
    data_original = fun_load_file(subfolder_path="..\\..\\01_data\\" + folder, name=file_name + ".xlsx")
    
    # Start feature "Instance_id" with one not zero and select columns that will not be changed
    data_original["Instance_id"] +=1
    data = data_original[["X", "Y", "Marginal_Cost", "Savings", "Shapley Value", "Total cost", "Number Customers", "X_Depot", "Y_Depot", 
                          "Instance_id", "Demand", "Vehicle Capacity", "Total demand"]]
    
    # Modify column names
    data = data.rename(columns={"Marginal_Cost": "Marginal Cost", "X_Depot": "X Depot", "Y_Depot": "Y Depot", "Total cost": "Total Cost", 
                                "Instance_id": "Instance ID", "Total demand": "Total Demand"})
    
    # Reorder columns
    columns = ["Instance ID", "Number Customers", "X", "Y", "Vehicle Capacity", "Demand", "Total Demand", "X Depot", "Y Depot", "Savings", 
               "Marginal Cost", "Total Cost", "Shapley Value"]
    data = data[columns]

# Print number of instances
print("Number of instances:", len(np.unique(data["Instance ID"])))

# Get a list of all instance sizes and the number of instances per size
num_customers_list = np.unique(data["Number Customers"])
number_of_instances_per_size = int(len(data[data["Number Customers"] == num_customers_list[0]]) / num_customers_list[0])
print("Instance sizes: {}\nNumber of instances per size: {}".format(num_customers_list, number_of_instances_per_size))

# Get the min and max vehicle capacity and customer demand for the CVRP
if (routing_problem == "CVRP"):
    min_capa = np.min(data["Vehicle Capacity"])
    max_capa = np.max(data["Vehicle Capacity"])
    min_demand = np.min(data["Demand"])
    max_demand = np.max(data["Demand"])
    print(f"Min/Max vehicle capacity: ({min_capa}, {max_capa})")
    print(f"Min/Max customer demand: ({min_demand}, {max_demand})")
display(data)

# Create a DataFrame with previous cluster features of Johannes
cluster_features_j = ["cluster", "customers_in_same_cluster", "distance_to_cluster_center", # Feature distance_to_cluster_center_ratio fehlt im Datensatz
                      "min_distance_to_other_cluster", "min_distance_to_other_cluster_ratio", "cluster_distance_to_depot", "cluster_distance_to_depot_ratio", 
                      "Cluster Area", "Cluster Area ratio"]
df_clusters_johannes = data_original[cluster_features_j]
df_clusters_johannes.columns = [string + str("_j") for string in cluster_features_j]

Number of instances: 12000
Instance sizes: [ 5  6  7  8  9 10 11 12]
Number of instances per size: 1500
Min/Max vehicle capacity: (10, 18)
Min/Max customer demand: (1, 5)


,Instance ID,Number Customers,X,Y,Vehicle Capacity,Demand,Total Demand,X Depot,Y Depot,Savings,Marginal Cost,Total Cost,Shapley Value
0,1,5,86.027966,95.446677,16,5,19,29.551338,5.437034,3.549781,53.092141,290.561690,85.337134
1,1,5,92.299886,89.092199,16,2,19,29.551338,5.437034,7.240922,7.240922,290.561690,72.157370
2,1,5,61.404822,7.793238,16,4,19,29.551338,5.437034,9.927915,50.239351,290.561690,30.779912
3,1,5,57.101808,4.267994,16,4,19,29.551338,5.437034,1.197420,50.739781,290.561690,28.850226
4,1,5,67.005710,89.535835,16,4,19,29.551338,5.437034,5.720845,55.263206,290.561690,73.437048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101995,12000,12,62.573873,51.000100,14,4,41,91.032404,49.408248,0.870624,18.862987,546.952173,23.357590
101996,12000,12,90.838056,93.465110,14,4,41,91.032404,49.408248,36.066497,57.921105,546.952173,60.120348
101997,12000,12,14.800099,38.550679,14,5,41,91.032404,49.408248,52.243572,77.151205,546.952173,72.730610
101998,12000,12,66.502247,28.687865,14,5,41,91.032404,49.408248,27.135692,27.135692,546.952173,32.926985


In [4]:
###############################################################################
# Generate random instances - settings
###############################################################################
# Define the range for X and Y coordinates and create DataFrame to merge final instances
x_range = (0, 100)
y_range = (0, 100)
df_final = pd.DataFrame()
run_time = {}
prints = True # Show prints and interim results during generating
max_instance_id = np.inf # Set parameter to "np.inf" to create features for all instances
old_cluster_features = False # Parameter to add previous cluster features of Johannes or not

###############################################################################
# START GENERAITING
###############################################################################
for instance_id in tqdm(np.unique(data["Instance ID"]), desc="Processing", unit="iteration", leave=True):

    if (instance_id % number_of_instances_per_size == 1): start = time.time() # Start time count if a new instance sice starts
    if (max_instance_id > 1): prints = False # No prints for more than one instance
    instance_df = data[data["Instance ID"] == instance_id]
    num_customers = int(instance_df.iloc[0]["Number Customers"])

    # Add depot with X and Y coordinates as first row (like in the original setting from function generate_tsp_instance)
    x_depot = instance_df.iloc[0]["X Depot"] # X coordinate of depot
    y_depot = instance_df.iloc[0]["Y Depot"] # Y coordinate of depot
    df_depot = pd.DataFrame(data={"Instance ID": instance_id, "Number Customers": num_customers, 
                                  "X": x_depot, "Y": y_depot}, index=[0])
    instance_df = pd.concat([df_depot, instance_df], ignore_index=True)[columns] # Index starts at zero for all instance_df

    if (prints == True): print("############### INSTANCE ###############"); display(instance_df)
    
    ###############################################################################
    # Add distance features
    ###############################################################################

    # Calculate the distance between the depot and each location
    instance_df["Depot Distance"] = np.sqrt((instance_df["X"] - x_depot) ** 2 + (instance_df["Y"] - y_depot) ** 2) # Euclidean distance

    # Calculate distances from each customer to all other customers (excluding depot) - customer distances matrix
    customer_distances = cdist(instance_df.iloc[1:][["X", "Y"]], instance_df.iloc[1:][["X", "Y"]], metric="euclidean")

    # Replace diagonal values with np.nan; otherwise the customer will be his own closest customer
    np.fill_diagonal(customer_distances, np.nan)

    # Create an array with indices of all customers and loop over each of them (excluding the depot)
    all_customers = np.arange(start=1, stop=num_customers+1)
    for i, customer in enumerate(all_customers): # i starts with zero, customer starts with one

        # Sort the distances for the current customer
        sorted_distances = np.sort(customer_distances[i])

        # Add columns for the distances to the closest, second closest, third closest, and fourth closest customers and add the respective values of the customer
        instance_df.loc[customer, "Closest Customer Distance (CCD)"] = sorted_distances[0]
        instance_df.loc[customer, "2nd CCD"] = sorted_distances[1]
        instance_df.loc[customer, "3rd CCD"] = sorted_distances[2]
        instance_df.loc[customer, "4th CCD"] = sorted_distances[3]
        instance_df.loc[customer, "5th CCD"] = sorted_distances[4] if (num_customers >= 6) else 100000
        instance_df.loc[customer, "6th CCD"] = sorted_distances[5] if (num_customers >= 7) else 100000
        instance_df.loc[customer, "7th CCD"] = sorted_distances[6] if (num_customers >= 8) else 100000
        instance_df.loc[customer, "8th CCD"] = sorted_distances[7] if (num_customers >= 9) else 100000
        instance_df.loc[customer, "9th CCD"] = sorted_distances[8] if (num_customers >= 10) else 100000
        instance_df.loc[customer, "10th CCD"] = sorted_distances[9] if (num_customers >= 11) else 100000

    # Calculate the mean distance to other customers
    customer_distances = pd.DataFrame(customer_distances, index=all_customers, columns=all_customers)
    mean_distances = customer_distances.mean(axis=1, skipna=True) # Calculate mean for each row over all columns, ignoring the np.nan

    # Add the "Mean distance to other customers" column to the DataFrame (We add a NaN value for the depot since it doesn't have a mean distance)
    instance_df["Mean Distance To Other Customers"] = [np.nan] + mean_distances.tolist()
    
    # Calculate for each customer the distance to centroid of all customrs (excluding the depot) and save it in a new column
    centroid_x = instance_df.loc[1:, "X"].mean()
    centroid_y = instance_df.loc[1:, "Y"].mean()
    instance_df["Gravity Center Distance"] = np.sqrt((instance_df.loc[1:, "X"] - centroid_x) ** 2 + (instance_df.loc[1:, "Y"] - centroid_y) ** 2)
    
    # Define columns for displaying the instance
    instance_features = ["Instance ID", "Number Customers", "X", "Y", "X Depot", "Y Depot"]
    if (routing_problem == "CVRP"): instance_features[4:4] = ["Vehicle Capacity", "Demand", "Total Demand"] # Insert these columns at location 4

    distance_features = ["Depot Distance", "Closest Customer Distance (CCD)", "2nd CCD", "3rd CCD", "4th CCD", "5th CCD", "6th CCD", 
                         "7th CCD", "8th CCD", "9th CCD", "10th CCD", "Mean Distance To Other Customers", "Gravity Center Distance"]
    
    if (prints == True):
        print("\n############### DISTANCE FEATURES ###############")
        print("Customer distances:"); display(customer_distances, instance_df[instance_features[:7] + distance_features])

    ###############################################################################
    # Add cluster features
    ###############################################################################

    # Cluster customers (without depot)
    # Get X and Y coordinates of the customers (excluding the depot)
    X = instance_df[["X", "Y"]][1:]

    # Clustering the customers of the instance
    assignments, core_point_indices = fun_multi_dbscan(X=X, num_customers=num_customers, prints=prints)

    # Extract X and Y coordinates of depot and all customers (first two columns) as tuples and give it as input for plot_instance
    coord = [(instance_df["X"][i], instance_df["Y"][i]) for i in range(len(instance_df))]
    
    # View clusters
    if (prints == True):
        print("\n############### CLUSTER CUSTOMERS ###############")
        plot_instance(coord=coord, sequence=None, total_cost=None, x_range=x_range, y_range=y_range, 
                 assignments=assignments, core_point_indices=core_point_indices, plot_sequence=False, routing_problem=routing_problem)

    # Define columns for displaying the instance
    cluster_features = ["Cluster", "Core Point", "Outlier", "Number Clusters", "Number Outliers", "Cluster Size", "X Centroid", "Y Centroid", "Centroid Distance", 
                        "Centroid Distance To Depot", "Distance To Closest Other Cluster", "Distance To Closest Other Centroid", "Cluster Area", "Cluster Density"]

    if (routing_problem == "CVRP"): cluster_features += ["Cluster Demand", "Cluster Demand Proportion"]

    # Execute function for cluster features
    instance_df = fun_cluster_features(data=instance_df, assignments=assignments, core_point_indices=core_point_indices, 
                                       features=instance_features[2:6]+cluster_features, prints=prints)
    
    ###############################################################################
    # Add statistics as features
    ###############################################################################

    # Extract X and Y values of the customers, excluding the first row (depot)
    x_customers = instance_df["X"].iloc[1:]
    y_customers = instance_df["Y"].iloc[1:]

    # Compute statistics for X and Y coordinates of the customers and add each of them as a column
    for statistic in [np.mean, np.std, np.max, np.min]:
        instance_df["X " + statistic.__name__.capitalize()] = statistic(x_customers)
        instance_df["Y " + statistic.__name__.capitalize()] = statistic(y_customers)

    # Calculate the correlation between X and Y of the customers as well as the skewness for X and Y and add statistics to each row as a column
    instance_df["Correlation"] = np.corrcoef(x_customers, y_customers)[0, 1] # Select first row and second column of correlation matrix
    instance_df["Skewness X"] = skew(x_customers)
    instance_df["Skewness Y"] = skew(y_customers)

    # Reorder columns
    last_columns = ["Savings", "Marginal Cost", "Total Cost", "Shapley Value"]
    if (routing_problem == "TSP"): last_columns += ["SHAPO", "Percentage Error"]

    instance_df = instance_df[[column for column in instance_df.columns if column not in last_columns] + last_columns]

    # Define columns for displaying the instance
    statistical_features = ["X Mean", "Y Mean", "X Std", "Y Std", "X Max", "Y Max", "X Min", "Y Min", "Correlation", "Skewness X", "Skewness Y"]

    if (prints == True): print("\n############### STATISTICAL FEATURES ###############"); display(instance_df[instance_features + statistical_features])

    ##################################################################################################################################################################
    # Solve the instance here and add features Savings, Marginal Cost, Total Cost (and Shapley Value) if necessary
    #
    #
    #
    ##################################################################################################################################################################

    ###############################################################################
    # Add ratio features
    ###############################################################################

    # Define features for which the ratio should be computed
    ratio_features = ["X", "Y", "Depot Distance", "Closest Customer Distance (CCD)", "2nd CCD", "3rd CCD", "4th CCD", "5th CCD", "6th CCD", "7th CCD", "8th CCD", 
                      "9th CCD", "10th CCD", "Mean Distance To Other Customers", "Gravity Center Distance", "Centroid Distance", "Centroid Distance To Depot", 
                      "Distance To Closest Other Cluster", "Distance To Closest Other Centroid", "Cluster Area", "Cluster Density", "Savings", "Marginal Cost"]

    if (routing_problem == "CVRP"): ratio_features += ["Demand", "Cluster Demand", "Cluster Demand Proportion"]

    # Compute mean of all ratio_features (without the depot) and divide their values by mean to obtain ratio column which is inserted before the Shapley value column
    for feature in ratio_features:
        mean = np.mean(instance_df.loc[1:, feature])
        ratio = instance_df.loc[1:, feature] / mean
        instance_df.insert(loc=len(instance_df.columns) - len(last_columns) + 3, column=feature + str(" Ratio"), value=ratio)

    if (prints == True):
        print("\n############### RATIO FEATURES ###############")
        print("No ratio features:\n", np.sort(list(set(instance_df.columns) - set(ratio_features) - set([i + str(" Ratio") for i in ratio_features]))))
        display(instance_df[[i + str(" Ratio") for i in ratio_features]])        
    
    # Drop depot row and reset index
    instance_df.drop(index=0, inplace=True)
    instance_df.reset_index(drop=True)

    # Merge instances
    df_final = pd.concat([df_final, instance_df], ignore_index=True)

    # Stop time count for instance size
    if (instance_id % number_of_instances_per_size == 0): run_time[num_customers] = time.time() - start

    # View final instance when there was only one modified instance
    if (max_instance_id == 1):
        print("\n############### FINAL INSTANCE ###############"); display(instance_df)
        plot_instance(coord=coord, sequence=None, total_cost=None, x_range=x_range, y_range=y_range, 
                      assignments=assignments, core_point_indices=core_point_indices, plot_sequence=True, print_sequence=True, routing_problem=routing_problem)

    if (instance_id == max_instance_id): break

# View run times and final DataFrame with merged instances
if (max_instance_id != 1):
    print("-> Total run time: ", fun_convert_time(seconds=sum(run_time.values())))
    display({"Instance size: {} run time".format(key): fun_convert_time(seconds=value) for key, value in run_time.items()})
    display(df_final)

# Merge final DataFrame and DataFrame with previous cluster features from Johannes if parameter is True
if (old_cluster_features == True): df_final = pd.merge(left=df_final, right=df_clusters_johannes, how="left", left_index=True, right_index=True)

# Save file
fun_save_file(data=df_final, subfolder_path="..\\..\\01_data\\" + folder, name=file_name + "_updated.xlsx")

Processing: 100%|██████████| 12000/12000 [22:41<00:00,  8.81iteration/s]

-> Total run time:  22m, 41s


{'Instance size: 5 run time': '2m, 42s',
 'Instance size: 6 run time': '2m, 38s',
 'Instance size: 7 run time': '2m, 38s',
 'Instance size: 8 run time': '2m, 46s',
 'Instance size: 9 run time': '2m, 52s',
 'Instance size: 10 run time': '2m, 55s',
 'Instance size: 11 run time': '3m, 5s',
 'Instance size: 12 run time': '3m, 1s'}

,Instance ID,Number Customers,X,Y,Vehicle Capacity,Demand,Total Demand,X Depot,Y Depot,Depot Distance,...,Distance To Closest Other Cluster Ratio,Distance To Closest Other Centroid Ratio,Cluster Area Ratio,Cluster Density Ratio,Savings Ratio,Marginal Cost Ratio,Demand Ratio,Cluster Demand Ratio,Cluster Demand Proportion Ratio,Shapley Value
0,1,5,86.027966,95.446677,16.0,5.0,19.0,29.551338,5.437034,106.260742,...,1.064303,1.053179,1.635119,0.101892,0.642218,1.225720,1.315789,1.122449,1.136364,85.337134
1,1,5,92.299886,89.092199,16.0,2.0,19.0,29.551338,5.437034,104.573261,...,1.016669,1.008670,1.635119,0.101892,1.310010,0.167169,0.526316,1.122449,0.454545,72.157370
2,1,5,61.404822,7.793238,16.0,4.0,19.0,29.551338,5.437034,31.940509,...,0.957786,0.970516,0.047321,2.347162,1.796135,1.159858,1.052632,0.816327,1.250000,30.779912
3,1,5,57.101808,4.267994,16.0,4.0,19.0,29.551338,5.437034,27.575262,...,1.003456,1.021359,0.047321,2.347162,0.216634,1.171411,1.052632,0.816327,1.250000,28.850226
4,1,5,67.005710,89.535835,16.0,4.0,19.0,29.551338,5.437034,92.062143,...,0.957786,0.946276,1.635119,0.101892,1.035002,1.275842,1.052632,1.122449,0.909091,73.437048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101995,12000,12,62.573873,51.000100,14.0,4.0,41.0,91.032404,49.408248,28.503018,...,0.744415,0.607843,0.618233,1.103139,0.053880,0.457787,1.170732,0.878049,1.333333,23.357590
101996,12000,12,90.838056,93.465110,14.0,4.0,41.0,91.032404,49.408248,44.057291,...,1.298501,1.255435,1.940459,0.351462,2.232051,1.405690,1.170732,0.878049,1.333333,60.120348
101997,12000,12,14.800099,38.550679,14.0,5.0,41.0,91.032404,49.408248,77.001631,...,1.116170,1.093645,0.441308,1.545399,3.233204,1.872386,1.463415,1.243902,1.176471,72.730610
101998,12000,12,66.502247,28.687865,14.0,5.0,41.0,91.032404,49.408248,32.110168,...,0.977961,0.947355,0.618233,1.103139,1.679350,0.658557,1.463415,0.878049,1.666667,32.926985


File saved successfully!
